In [1]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

# 2.4. Обучение и проверка моделей
Обучите Vowpal Wabbit на выборке stackoverflow_train.vw 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3). Остальные параметры укажите следующие: bit_precision=28 и seed=17. Также скажите VW, что это 10-классовая задача.
Проверяйте долю правильных ответов на выборке stackoverflow_valid.vw. Выберите лучшую модель и проверьте качество на выборке stackoverflow_test.vw.

 -oaa 10 – указываем, что классификация на 10 классов

-d – путь к данным
-f – путь к модели, которая будет построена
-b 28 – используем 28 бит для хэширования, то есть признаковое пространство ограничено 228 признаками, что в данном случае больше, чем число уникальных слов в выборке (но потом появятся би- и триграммы, и ограничение размерности признакового пространства начнет работать)
также указываем random seed
--quiet

In [ ]:
%%time
!vw --oaa 10 -d /ml/stackoverflow_train.vw -f /ml/vw_model1_10mln.vw -b28 --random_seed 17 --passes=5 --ngram=3 -c

Generating 3-grams for all namespaces.
final_regressor = /ml/vw_model1_10mln.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = /ml/stackoverflow_train.vw.cache
Reading datafile = /ml/stackoverflow_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0  unknown        1      475
1.000000 1.000000            2            2.0  unknown        1      196
1.000000 1.000000            4            4.0  unknown        1      256
1.000000 1.000000            8            8.0  unknown        1      277
1.000000 1.000000           16           16.0  unknown        1      469
1.000000 1.000000           32           32.0  unknown        1     1204
1.000000 1.000000           64           64.0  unknown        1      301
1.000000 1.000000          128          128.0  unknow